In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler

In [3]:
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

In [83]:
Cough_dataframe = pd.read_csv("./cough.csv")
Wheeze_dataframe = pd.read_csv("./Wheeze.csv")
Aaa_dataframe = pd.read_csv("./Aaa.csv")
Yee_dataframe = pd.read_csv("./Yee.csv")
Inhale_dataframe = pd.read_csv("./Inhale.csv")
Exhale_dataframe = pd.read_csv("./Exhale.csv")
Ooo_dataframe = pd.read_csv("./Ooo.csv")
Sss_dataframe = pd.read_csv("./Sss.csv")
Eee_dataframe = pd.read_csv("./Eee.csv")
Uuu_dataframe = pd.read_csv("./Uuu.csv")
Zzz_dataframe = pd.read_csv("./Zzz.csv")

In [17]:
def fold(sound_dataframe, sets_loc):
    big = []
    for j in os.listdir(sets_loc):      

        file = open('./SETS/'+j, 'r') 
        Lines = file.readlines() 

        count = 0
        # Strips the newline character 
        # for line in Lines: 
        #     print("Line{}: {}".format(count, line.strip())) 

        set_dataframe = pd.DataFrame()
        for i, line in enumerate(Lines):
            A = sound_dataframe[(sound_dataframe['217'] == (Lines[i].strip()+".mat"))]
            set_dataframe = pd.DataFrame.append(set_dataframe,A)
        
        big.append(set_dataframe)
    fold_1 = shuffle(pd.concat(big[:4]))
    test_1 = shuffle(big[4])
    fold_2 = shuffle(pd.concat(big[1:5]))
    test_2 = shuffle(big[0])
    fold_3 = shuffle(pd.concat([big[0],big[2],big[3],big[4]]))
    test_3 = shuffle(big[1])
    fold_4 = shuffle(pd.concat([big[0],big[1],big[3],big[4]]))
    test_4 = shuffle(big[2])
    fold_5 = shuffle(pd.concat([big[0],big[2],big[1],big[4]]))
    test_5 = shuffle(big[3])
    
    return fold_1,test_1,fold_2,test_2,fold_3,test_3,fold_4,test_4,fold_5,test_5

In [ ]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Cough_dataframe,sets_loc)

In [27]:
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]

In [47]:
def SVM(fold_list):
    list_train = []
    list_test = []
    for i in range(0,len(fold_list),2):
        fold = fold_list[i]
        test = fold_list[i+1]
        X_train = fold.iloc[:,:216]
        y_train = fold.iloc[:,216]
        X_test = test.iloc[:,:216]
        y_test = test.iloc[:,216]


        scaler = StandardScaler()

        X_train = scaler.fit_transform(X_train)

        X_test = scaler.transform(X_test)

        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)



        ### SVM

        svm = SVC()

        svm.fit(X_train,y_train)

        list_test.append(accuracy_score(y_test,svm.predict(X_test)))

        list_train.append(accuracy_score(y_train,svm.predict(X_train)))
        
    return list_test, list_train

In [49]:
cough_test, cough_train = SVM(fold_list)

In [54]:
from sklearn.model_selection import StratifiedShuffleSplit

In [98]:
def hyperparameter_tuning(fold,test):
    X_train = fold.iloc[:,:216]
    y_train = fold.iloc[:,216]    
    C_range = np.logspace(-1, 10, 12)
    gamma_range = np.logspace(-1, 20, 22)
    param_grid = dict(gamma=gamma_range, C=C_range)
    cv = StratifiedShuffleSplit(n_splits=5)
    grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv, verbose=3, n_jobs=-1)
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    y_pred = grid.predict(test.iloc[:,:216])
    a = accuracy_score(test.iloc[:,216],y_pred)
    return a


In [70]:
tune_acc_cough = []
for i in range(0,len(fold_list),2):
    tune_acc_cough.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   31.8s
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:   46.5s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   53.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:   36.8s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   42.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:   43.3s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   50.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   29.7s
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:   42.1s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   48.6s finished


In [141]:
np.mean(cough_test)

0.6297472258513764

In [137]:
(tune_acc_cough)

[0.7448275862068966,
 0.6029411764705882,
 0.6627906976744186,
 0.5849056603773585,
 0.6606060606060606]

In [138]:
np.mean(tune_acc_cough)

0.6512142362670645

In [144]:
a = [1,2,3,4]

In [146]:
np.std(tune_acc_cough)

0.05605853306807637

### Wheeze

In [76]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Wheeze_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
wheeze_test, wheeze_train = SVM(fold_list)


In [77]:
wheeze_test, wheeze_train

([0.7217391304347827,
  0.6423357664233577,
  0.7301587301587301,
  0.8223684210526315,
  0.6062992125984252],
 [0.974169741697417,
  0.9826923076923076,
  0.9830508474576272,
  0.9722772277227723,
  0.9679245283018868])

In [91]:
np.mean(wheeze_test)

0.7045802521335854

In [166]:
np.std(wheeze_test)

0.07530773813278728

In [78]:
tune_acc_wheeze = []
for i in range(0,len(fold_list),2):
    tune_acc_wheeze.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   30.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   25.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   24.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   19.5s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   26.2s finished


In [79]:
tune_acc_wheeze

[0.6608695652173913,
 0.7299270072992701,
 0.6825396825396826,
 0.7171052631578947,
 0.7007874015748031]

In [80]:
np.mean(tune_acc_wheeze)

0.6982457839578083

In [81]:
np.std(tune_acc_wheeze)

0.02453366274044316

### Aaa 

In [84]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Aaa_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Aaa_test, Aaa_train = SVM(fold_list)


In [90]:
np.mean(Aaa_test), Aaa_train

(0.530039687599836,
 [0.953551912568306,
  0.9545454545454546,
  0.9486486486486486,
  0.9700272479564033,
  0.9439775910364145])

In [86]:
tune_acc_Aaa = []
for i in range(0,len(fold_list),2):
    tune_acc_Aaa.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 175 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 495 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 943 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 1305 out of 1320 | elapsed:   14.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   14.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   12.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   12.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:   10.1s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   12.7s finished


In [147]:
np.mean(tune_acc_Aaa)

0.585253578467337

In [148]:
tune_acc_Aaa

[0.4942528735632184, 0.57, 0.6588235294117647, 0.65, 0.5531914893617021]

In [149]:
np.std(tune_acc_Aaa)

0.0618811995523263

### Eee

In [87]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Eee_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Eee_test, Eee_train = SVM(fold_list)


In [93]:
Eee_test, Eee_train

([0.632183908045977, 0.61, 0.6823529411764706, 0.6875, 0.5851063829787234],
 [0.9888579387186629,
  0.9826589595375722,
  0.9833795013850416,
  0.9808743169398907,
  0.9886363636363636])

In [94]:
np.mean(Eee_test)

0.6394286464402342

In [99]:
tune_acc_Eee = []
for i in range(0,len(fold_list),2):
    tune_acc_Eee.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 177 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 497 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 945 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 1305 out of 1320 | elapsed:   13.9s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   14.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   11.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   13.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:    9.8s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   12.6s finished


In [150]:
tune_acc_Eee

[0.4827586206896552, 0.61, 0.6588235294117647, 0.65, 0.5531914893617021]

In [102]:
np.mean(tune_acc_Eee)

0.5909547278926244

In [153]:
np.std(Eee_test)

0.04005693419361653

### Yee

In [104]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Yee_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Yee_test, Yee_train = SVM(fold_list)

In [105]:
Yee_test, Yee_train

([0.6627906976744186,
  0.648936170212766,
  0.6091954022988506,
  0.5921052631578947,
  0.4883720930232558],
 [0.9825072886297376,
  0.991044776119403,
  0.9883040935672515,
  0.9801699716713881,
  0.9854227405247813])

In [106]:
np.mean(Yee_test)

0.6002799252734372

In [107]:
tune_acc_Yee = []
for i in range(0,len(fold_list),2):
    tune_acc_Yee.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 281 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 921 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 1305 out of 1320 | elapsed:   12.7s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   12.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   11.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   11.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    5.2s


{'C': 1.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   11.3s finished


In [154]:
tune_acc_Yee

[0.5116279069767442,
 0.5319148936170213,
 0.5632183908045977,
 0.5921052631578947,
 0.5]

In [155]:
np.mean(tune_acc_Yee)

0.5397732909112516

In [157]:
np.mean(Yee_test)


0.6002799252734372

In [158]:
np.std(Yee_test)

0.06155436716918134

### Inhale

In [108]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Inhale_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Inhale_test, Inhale_train = SVM(fold_list)
tune_acc_Inhale = []
for i in range(0,len(fold_list),2):
    tune_acc_Inhale.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   36.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   32.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   33.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   26.9s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   36.0s finished


In [159]:
np.mean(tune_acc_Inhale)

0.7095341084910862

In [160]:
np.mean(Inhale_test)

0.6476936378203118

In [163]:
tune_acc_Inhale

[0.8137931034482758,
 0.6134969325153374,
 0.7007299270072993,
 0.7317073170731707,
 0.6879432624113475]

In [164]:
np.std(tune_acc_Inhale)

0.0650052407483324

### Exhale

In [109]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Exhale_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Exhale_test, Exhale_train = SVM(fold_list)
tune_acc_Exhale = []
for i in range(0,len(fold_list),2):
    tune_acc_Exhale.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   25.8s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   34.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   32.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   23.7s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   31.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   25.5s


{'C': 1.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   34.3s finished


In [121]:
np.mean(Exhale_test)

0.7010783899637794

In [118]:
tune_acc_Exhale

[0.7638888888888888,
 0.7361963190184049,
 0.8088235294117647,
 0.7987804878048781,
 0.6099290780141844]

In [119]:
np.mean(tune_acc_Exhale)

0.7435236606276241

In [165]:
np.std(tune_acc_Exhale)

0.07160074688684982

### Ooo

In [110]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Ooo_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Ooo_test, Ooo_train = SVM(fold_list)
tune_acc_Ooo = []
for i in range(0,len(fold_list),2):
    tune_acc_Ooo.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   11.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   11.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   13.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    5.4s


{'C': 1.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   12.1s finished


In [125]:
np.mean(Ooo_test)

0.5852947741925505

In [169]:
Ooo_test

[0.6224489795918368,
 0.6161616161616161,
 0.5238095238095238,
 0.5789473684210527,
 0.5851063829787234]

In [170]:
np.std(Ooo_test)

0.03508263951853279

In [124]:
np.mean(tune_acc_Ooo)

0.5034366103155097

### Sss

In [112]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Sss_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Sss_test, Sss_train = SVM(fold_list)
tune_acc_Sss = []
for i in range(0,len(fold_list),2):
    tune_acc_Sss.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    9.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    9.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.5s


{'C': 1.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   10.0s finished


In [129]:
np.mean(Sss_test)

0.6177101852522553

In [173]:
tune_acc_Sss

[0.5,
 0.6170212765957447,
 0.5584415584415584,
 0.7027027027027027,
 0.684931506849315]

In [128]:
np.mean(tune_acc_Sss)

0.6126194089178643

In [174]:
np.std(tune_acc_Sss)

0.07613381182190852

### Uuu

In [111]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Uuu_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Uuu_test, Uuu_train = SVM(fold_list)
tune_acc_Uuu = []
for i in range(0,len(fold_list),2):
    tune_acc_Uuu.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   11.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   11.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   11.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    5.3s


{'C': 1.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   11.3s finished


In [167]:
Uuu_test

[0.6329113924050633,
 0.5227272727272727,
 0.5058823529411764,
 0.6,
 0.7653061224489796]

In [132]:
np.mean(Uuu_test)

0.6053654281044984

In [168]:
np.std(Uuu_test)

0.09283807477914321

In [131]:
np.mean(tune_acc_Uuu)

0.5506456587237888

### Zzz

In [113]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Zzz_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Zzz_test, Zzz_train = SVM(fold_list)
tune_acc_Zzz = []
for i in range(0,len(fold_list),2):
    tune_acc_Zzz.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    7.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    7.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    7.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    5.0s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.4s finished


In [136]:
np.mean(Zzz_test)

0.452967354514685

In [171]:
tune_acc_Zzz

[0.4647887323943662, 0.43023255813953487, 0.5833333333333334, 0.48, 0.45]

In [135]:
np.mean(tune_acc_Zzz)

0.48167092477344686

In [172]:
np.std(tune_acc_Zzz)

0.0534265828240777